In [8]:
import sys, os
module_path = os.path.abspath(os.getcwd()+'/../..')
print(sys.path)
print(module_path)

sys.path.append(module_path)

['/opt/conda/envs/green.ai/lib/python311.zip', '/opt/conda/envs/green.ai/lib/python3.11', '/opt/conda/envs/green.ai/lib/python3.11/lib-dynload', '', '/opt/conda/envs/green.ai/lib/python3.11/site-packages', '/opt/conda/envs/green.ai/lib/python3.11/site-packages/setuptools/_vendor', '/var/tmp/tmpwovsk3yb', '/home/jupyter/green.ai/experiments', '/home/jupyter/green.ai/experiments', '/home/jupyter/green.ai/experiments', '/home/jupyter/green.ai/experiments']
/home/jupyter/green.ai


In [2]:
from pydantic import Field
from deepeval.dataset import Golden
from deepeval.test_case import LLMTestCase

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from benchmarq.utility import Evaluator
from benchmarq.experiment import Experiment

from dotenv import load_dotenv

load_dotenv()

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

# noinspection PyPep8Naming
def generate(prompt: str) -> str:
    return generator(prompt)


class Test(Evaluator):
    def __init__(self):
        super().__init__()

    def evaluate_consumption(self, input: Golden):
        generate(input.input)

    def evaluate_test_case(self, input: Golden) -> LLMTestCase:
        output = generate(f"{input.context}: {input.input}")[0]["generated_text"]
        return LLMTestCase(input=input.input, expected_output=input.expected_output, actual_output=output, context=input.context, retrieval_context=input.retrieval_context)

experiment = Experiment(
    subquestion_id="test_1",
    id="case_1",
    subquestion_path="experiments/test/tests.json",
    name="name",
    description="A very long description",
    settings=Test())

a=experiment.run()


/opt/conda/envs/green.ai/lib/python3.11/site-packages/deepeval/__init__.py:54: UserWarning: You are using deepeval version 2.5.7, however version 2.5.8 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


ModuleNotFoundError: No module named 'benchmarq'

In [ ]:
import os
import pandas as pd

a=pd.read_csv("/Users/leenlaptop/Documents/repos/greenai/green.ai/experiments/test/inputs.csv")

print(a)

In [2]:
experiment.create_subquestion_json()

{'subquestion_id': 'test',
 'subquestion_metrics_path': 'experiments/test/tests.json',
 'experiments': [{'id': '8c30631fad934d5da404ad35ac98468e',
   'name': 'name',
   'description': 'A very long description',
   'settings': {},
   'runs': [{'consumption_results': {'cloud_provider': '',
      'cloud_region': '',
      'codecarbon_version': '2.2.2',
      'country_iso_code': 'NLD',
      'country_name': 'The Netherlands',
      'cpu_count': 16,
      'cpu_energy': 0.00010849921256303787,
      'cpu_model': 'Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz',
      'cpu_power': 22.5,
      'duration': 17.361709117889404,
      'emissions': 4.551006183701586e-05,
      'emissions_rate': 2.621289271004002e-06,
      'energy_consumed': 0.00013742910931507747,
      'experiment_id': '1',
      'gpu_count': None,
      'gpu_energy': 0,
      'gpu_model': None,
      'gpu_power': 0.0,
      'latitude': 51.5542,
      'longitude': 5.0661,
      'on_cloud': 'N',
      'os': 'macOS-15.3.1-x86_64-i386-64b

In [9]:
from benchmarq.utility import serve_vllm
 
serve_vllm(model="facebook/opt-125m")

True

INFO 03-18 08:29:05 __init__.py:207] Automatically detected platform cuda.


usage: vllm serve <model_tag> [options]
vllm serve: error: the following arguments are required: model_tag
